In [1]:
import numpy as np
import urllib.request as urllib
import os

import io
from PIL import Image

import matplotlib.pyplot as plt

cam2 = "http://orangepi:8080/video.mjpg"

stream=urllib.urlopen(cam2)

In [6]:
# %load receive.py
import asyncio
import asyncio_redis
import base64

def decoder(img_str):
    return base64.b64encode(img_str).decode('utf-8')

def encoder(img_str):
    return base64.b64decode(img_str.encode('utf-8'))

async def capture_stream(stream, bytes_=b'', chunksize=1024):
    while True:
        bytes_ += stream.read(chunksize)
        a = bytes_.find(b'\xff\xd8')
        b = bytes_.find(b'\xff\xd9')
        if a!=-1 and b!=-1 and a < b:
            jpg = bytes_[a:b+2]
            bytes_ = bytes_[b+2:]
            yield jpg
        
async def write_to_buffer(image, frame_no):
    conn_ = await asyncio_redis.Pool.create(host='redis', port=6379)
    f = 'frame_%05d' % frame_no
    if (frame_no % 1000) == 0:
        print(f)
    await conn_.set(f, decoder(image))
    conn_.close()

In [14]:
l = []
ll = []
lll = []
m = 0
async def t(n):
    m = 0
    async for i in capture_stream(stream):
        await write_to_buffer(i, m, connection)
        im = Image.open(io.BytesIO(i))
        l.append(im)
        lll.append(i)
        if m > n:
            break
        m += 1

In [16]:
import redis

In [17]:
r = redis.StrictRedis(host='redis')

In [29]:
a = r.get('frame_10000')

In [30]:
a = encoder(a.decode('utf-8'))
def enc(a):
    return encoder(a.decode('utf-8'))

In [44]:
print(len((lambda n: [(k, enc(r.get(k))) for i,k in enumerate(r.keys()) if i < n])(1)))
def framegen(n):
    for i,k in enumerate(r.keys()):
        if i < n:
            yield (k, enc(r.get(k)))

[(b'frame_01775', b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x0f\x00\x10\x00\x00\xff\xfe\x00\x10Lavc56.41.100\x00\xff\xdb\x00C\x00\x08ffyfy\x8c\x8c\x8c\x8c\x8c\x8c\xa5\x99\xa5\xac\xac\xac\xa5\xa5\xa5\xa5\xac\xac\xac\xb8\xb8\xb8\xd8\xd8\xd8\xb8\xb8\xb8\xac\xac\xb8\xb8\xcc\xcc\xd8\xd8\xeb\xf2\xeb\xdf\xdf\xd8\xdf\xf2\xf2\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xc4\x01\xa2\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x01\x00\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc

In [53]:
[frame[0].decode('utf-8') for frame in framegen(n=1)]

['frame_01775']

In [55]:
FRAMES = 10000
_ = [Image.open(io.BytesIO(f))
            .save(f"./image/from_buffer/{k}.png","PNG") 
     for k,f in ((key.decode('utf-8'), frame) for key, frame in framegen(n=FRAMES))]